In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from chbmit.chbmit import FilteredCMP, CHB_MIT_PAITENT, STFT
from mlutils import train

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
d = CHB_MIT_PAITENT

In [4]:
regression_data = FilteredCMP(512, 1, "/workspace/physionet.org/files/chbmit/1.0.0/chb01", 3600 * 2, 0, regression=True, use_tok_dim=False)
#classification_data = FilteredCMP(512, 1, "/workspace/physionet.org/files/chbmit/1.0.0/chb01", 3600 * 2, 0, regression=False, use_tok_dim=False)

  0%|          | 0/306 [00:00<?, ?it/s]

In [6]:
dl = torch.utils.data.DataLoader(regression_data, batch_size=8)
data, label = next(iter(dl))
m = torch.nn.Sequential(
          STFT(10 * 256, 10 * 256, device=device),
          torch.nn.Conv2d(23, 126, (3, 3)),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d((2, 2)),
          torch.nn.Conv2d(126, 64, (3, 3)),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d((2, 2)),
          torch.nn.Conv2d(64, 64, (3, 3)),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d((2, 2)),
          torch.nn.Flatten(),
          torch.nn.Linear(40448, 4096),
          torch.nn.ReLU(),
          torch.nn.Linear(4096, 256),
          torch.nn.ReLU(),
          torch.nn.Linear(256, 3),
        ).to(device)

s = STFT(10 * 256, 10 * 256, )
s.to(device)
m(data.to(device))

tensor([[-0.1634, -0.0225, -0.0458],
        [-0.1695, -0.0363, -0.0380],
        [-0.1593, -0.0271, -0.0370],
        [-0.1257, -0.0417, -0.0292],
        [-0.1940, -0.0261,  0.0351],
        [-0.1588, -0.0268, -0.0275],
        [-0.1544, -0.0197, -0.0296],
        [-0.1122, -0.0485, -0.0389]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

# 1D Conv Network

## Models and Init

In [6]:
class SimpleConv(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.feat_exctract = torch.nn.Sequential(torch.nn.Conv1d(23, 64, (5)),
                                             torch.nn.ReLU(),
                                             torch.nn.MaxPool1d((3)),
                                             torch.nn.Conv1d(64, 64, (5)),
                                             torch.nn.ReLU(),
                                             torch.nn.MaxPool1d((3)),
                                             torch.nn.Conv1d(64, 64, (5)),
                                             torch.nn.ReLU(),
                                             torch.nn.MaxPool1d((3)))
    self.class_head = torch.nn.Sequential(torch.nn.Linear(310528, 4096),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(4096, 128),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(128, 2))

  def forward(self, data):
    return self.class_head(self.feat_exctract(data).flatten())